In [ ]:
# Loading packages
using LiteQTL
using DelimitedFiles

In [ ]:
# Location of genotype file and phenotype file
geno_file = joinpath(@__DIR__, "..", "data", "processed", "spleen-bxd-genoprob.csv")
pheno_file = joinpath(@__DIR__, "..", "data","processed", "spleen-pheno-nomissing.csv")

# Whether to output whole LOD matrix or only maximum LOD of each phenptype? Default is false (only maximum)
export_matrix = false

# Location of output 
output_file = "output.csv"

# Setting BLAS threads as 16 threads. 
LiteQTL.set_blas_threads(16);

# Read in data.
G = LiteQTL.get_geno_data(geno_file, Float64)
Y = LiteQTL.get_pheno_data(pheno_file, Float64, transposed=false)
# getting geno and pheno file size.
n = size(Y,1) # number of individuals
m = size(Y,2) # number of markers
p = size(G,2) # number of phenotypes
println("******* Indivuduals n: $n, Traits m: $m, Markers p: $p ****************");
# run analysis. 
lod = LiteQTL.cpurun(Y, G,n,export_matrix);

# Run this function multiple times to get a better benchmarking of timing. 
cpu_timing = benchmark(10, cpurun, Y, G,n,export_matrix);
println("CPU timing: $(cpu_timing[3])")

# run the same function but with GPU: 
# lod = LiteQTL.gpurun(Y, G,n,m,p)

# write output to file
writedlm(joinpath(Base.@__DIR__, "..", "data", "results", output_file), lod, ',')

